# PyTorch Computer Vision



## 0. Computer Vision Libraries

* `torchvision` - base domain library
* `torchvision.datasets` - get datasets and data loading functions for computer vision
* `torchvision.models` - functions for manipulating your vision data (images) to be suitable for use with an ML model
* `torch.utils.data.Dataset` - Base dataset class for PyTorch
* `torch.utils.data.DataLoader` - Creates a Python iterable over a datset.

In [2]:
# Base PyTorch
import torch
from torch import nn

# Torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

# Vizualizations
import matplotlib.pyplot as plt

# Check versions
print(torch.__version__)
print(torchvision.__version__)


2.3.1
0.18.1


## 1. Getting a dataset

The data we will be using is FashionMNIST dataset, an advanced version of the classic MNIST, but on clothing instead of numbers.